                                        ####DEFINICIONES####

In [ ]:
from requests.exceptions import RequestException
import requests, os, time, sqlite3, random
from bs4 import BeautifulSoup
from urllib.parse import urljoin


os.system("cls")

url = 'https://books.toscrape.com/'
head = {
    'User-Agent':'MiSraper/1.0 (oscaaraujo96@gmail.com)'
}
session = requests.session()

                                        ####FUNCIONES####

In [ ]:
###Realiza la conexión y get de forma segura y con reintentos
def safe_get (url, intentos=3, delay=2):
    for i in range(intentos):
        try:
            respuesta = session.get(url, headers=head, timeout=10)
            respuesta.raise_for_status()
            return respuesta
        except RequestException as e:
            print(f"Intento {i+1}] Error en {url}: {e}")
            time.sleep(delay)
    return None


###Se obtiene las categorias del sitio
def obtener_categorias (url_base, db_cursor, db_conexion):

    respuesta = safe_get(url_base)
    soup = BeautifulSoup(respuesta.text, "html.parser")
    list_category = soup.select_one("ul.nav")
    data_category = list_category.select("ul li a")

    url_categorias = {}
    for link in data_category[1:]:
        url_category = link["href"]
        nombre_categoria= link.get_text(strip=True)
        url_categorias[nombre_categoria] = urljoin(url, url_category)

        db_cursor.execute("""
                INSERT OR IGNORE INTO categoria (name)
                VALUES (?)    
            """, (nombre_categoria,))
        db_conexion.commit()
    
    return url_categorias


###Scrap de una categoría
def scrap_categorie (url_categoria, nombre_categoria,db_cursor):
    all_books = []
    
    while True:
        
        resp = safe_get(url_categoria)
        soup = BeautifulSoup(resp.text, "html.parser")
        libros = soup.select("article.product_pod")
        for libro in libros:
            id_libro = 0
            titulo = libro.select_one("h3 a")["title"]
            tipo_moneda ,value = libro.select_one("p.price_color").text.split("£", 1)
            precio = float(value)
            
            _, rating_get = libro.select_one("p.star-rating").get("class", [])
            rating_map = {"One":1, "Two":2, "Three":3, "Four":4, "Five":5}
            rating = 0
            if rating_get in rating_map:
                rating += rating_map[rating_get]

            all_books.append({"titulo":titulo, "precio":precio, "puntuacion":rating})
            db_cursor.execute("SELECT id FROM categoria WHERE name = ?", (nombre_categoria,))
            id_categoria = db_cursor.fetchone()[0]
            db_cursor.execute("""
                INSERT INTO libros (name, precio, rating, categoria_id)
                VALUES (?,?,?,?)    
            """, (titulo, precio, rating, id_categoria))
        #buscar boton next
        next_tag = soup.select_one("li.next a")
        if next_tag:
            next_url = urljoin(url_categoria, next_tag["href"])
            url_categoria = next_url
            time.sleep(0.3)
        else:
            break
    return all_books

####Sracp de todas las categorías
def scrap_all_categories(url_categorias, db_cursor, db_conexion):
    for name,dir_categoria in url_categorias.items():
        scrap_categorie(dir_categoria,name, db_cursor)
        print(f"Categoria trabajada {name}")
        db_conexion.commit()

                                        ####DB-UML####

+---------------+        +----------------------+        +-----------------+
|  libros     	|        | libros_autores     	|        | autores         |
+---------------+        +----------------------+        +-----------------+
| id (PK)     	| 1   ∞  | libro_id (FK)       	| ∞   1  | id (PK)         |
| name        	|--------| autor_id (FK)     	|--------| name            |
| precio       	|        +----------------------+        +-----------------+
| rating      	|
| categoria_id 	|
+---------------+

+--------------+
| categoria   |
+--------------+
| id (PK)      |
| name         |
+--------------+

                                    ####CREATE DB AND TABLES####

In [ ]:
conexion = sqlite3.connect("books.db")
cursor = conexion.cursor()

###TABLA CATEGORIA
cursor.execute("""
    CREATE TABLE IF NOT EXISTS categoria (
        id INTEGER PRIMARY KEY AUTOINCREMENT,
        name TEXT NOT NULL
    )
""")

###TABLA LIBROS
cursor.execute("""
    CREATE TABLE IF NOT EXISTS libros (
        id INTEGER PRIMARY KEY AUTOINCREMENT,
        name TEXT NOT NULL ,
        precio REAL NOT NULL, 
        rating INTEGER NOT NULL,
        categoria_id TEXT NOT NULL
        
    )
""")

###TABLA AUTORES
cursor.execute("""
    CREATE TABLE IF NOT EXISTS autores (
        id INTEGER PRIMARY KEY AUTOINCREMENT,
        name TEXT UNIQUE NOT NULL
    );
""")

###TABLA LIBROS-AUTORES
cursor.execute("""
    CREATE TABLE IF NOT EXISTS libros_autores (
        libro_id INTEGER NOT NULL,
        autor_id INTEGER NOT NULL,
        PRIMARY KEY (libro_id, autor_id),
        FOREIGN KEY (libro_id) REFERENCES libros(id),
        FOREIGN KEY (autor_id) REFERENCES autores(id)
    );
""")

conexion.commit()


                                    SCRAP(Para poblar tabla BOOKS y CATEGORIAS)

In [ ]:
###Realiza INSERT EN TABLA CATEGORIAS
url_categorias = obtener_categorias(url, cursor, conexion)

####Realiza INSERT en tabla LIBROS
scrap_all_categories(url_categorias, cursor, conexion)

                                        Insert para poblar Autores

In [ ]:
# cursor.execute("""
#     INSERT INTO autores (name) VALUES
#     ('George Orwell'),
#     ('J.K. Rowling'),
#     ('Gabriel García Márquez'),
#     ('J.R.R. Tolkien'),
#     ('Jane Austen'),
#     ('Ernest Hemingway'),
#     ('Mark Twain'),
#     ('Haruki Murakami'),
#     ('Agatha Christie'),
#     ('Stephen King'),
#     ('Leo Tolstoy'),
#     ('F. Scott Fitzgerald'),
#     ('Isaac Asimov'),
#     ('Ray Bradbury'),
#     ('Mary Shelley'),
#     ('Arthur Conan Doyle'),
#     ('H. P. Lovecraft'),
#     ('Emily Brontë'),
#     ('Carlos Ruiz Zafón'),
#     ('Oscar Wilde')

# """)
##DELETE PARA PRUEBAS
cursor.execute('DELETE FROM libros_autores')
conexion.commit()

Para poblar tabla puente    

In [ ]:
# Se inserta autor aleatorio en libros una cantidad de 50 libros se actualizan
cantidad = 0
libro_id = []
while True:
    book_id = random.randint(1,1001)
    author_id = random.randint(1,21)

    if book_id not in libro_id:
        cursor.execute("""
            INSERT  INTO libros_autores (libro_id, autor_id)
            VALUES (?, ?)
        """, (book_id, author_id))
        conexion.commit()
        cantidad += 1
        libro_id.append(book_id)
    
    if cantidad == 1000:
        break

                                                        TEST INDICE

In [ ]:
import time
###TEST SIN INDICE
t0 = time.time()
cursor.execute("SELECT * FROM libros where name like 'Dune%'")
cursor.fetchall()
print("Sin índice:", time.time() - t0)

###SE CREA EL ÍNDICE
cursor.execute("CREATE INDEX idx_libros_name ON libros(name);")

###TEST CON INDICE
t1 = time.time()
cursor.execute("SELECT * FROM libros where name like 'Dune%'")
cursor.fetchall()
print("Con índice:", time.time() - t1)
cursor.fetchall()

###PARA PRUEBAS
# cursor.execute("DROP INDEX idx_libros_name;")

                                                    5 CONSULTAS SQL

In [ ]:
                                #####10 LIBROS POR PRECIO MENOR A X Y CATEGORIA MAYOR A X
cursor.execute("""
    SELECT name, precio FROM libros WHERE precio <= 50 and rating >= 3 limit 10;
""")
data =cursor.fetchall()

for a in data:
    print(a)

conexion.commit()

In [ ]:
                                #####10 LIBROS MAS CAROS                    
cursor.execute("""
    SELECT name, precio FROM libros ORDER BY precio DESC LIMIT 10;
""")
data =cursor.fetchall()

for a in data:
    print(a)

In [ ]:
                                #####CANTIDAD DE LIBROS POR CATEGORIAS (5 CATEGORIAS)                   
cursor.execute("""
    SELECT c.name, COUNT(b.id)
    FROM categoria c
    LEFT JOIN libros b ON b.categoria_id = c.id
    GROUP BY c.name LIMIT 5;

""")
data =cursor.fetchall()

for a in data:
    print(a)

In [ ]:
                                ##### 10 LIBROS DE MAYOR RATING                
cursor.execute("""
    SELECT name, rating FROM libros WHERE rating = 5 LIMIT 10;
""")
data =cursor.fetchall()

for a in data:
    print(a)

In [ ]:
                                ##### 10 LIBROS CON SU RESPECTIVO AUTOR              
cursor.execute("""
    SELECT b.name AS libro, a.name AS autor
    FROM libros b
    JOIN libros_autores ba ON ba.libro_id = b.id
    JOIN autores a ON a.id = ba.autor_id
    LIMIT 10;
""")
data =cursor.fetchall()

for a in data:
    print(a)

In [ ]:
                                                ###PRACTICA
# cursor.execute("""
#     SELECT autores.name, COUNT(libros_autores.autor_id)
#     FROM libros_autores
#     JOIN autores ON libros_autores.autor_id = autores.id
#     GROUP BY autores.name
#     HAVING COUNT(*) >= 4
# """)
# cursor.execute("""
#     SELECT name, precio
#     FROM libros
#     where precio > (SELECT AVG(precio) from libros)
#     ORDER BY PRECIO DESC
#     LIMIT 10
# """)
cursor.execute('''
    SELECT libros.name, autores.name
    from libros
    JOIN libros_autores ON libros.id = libros_autores.libro_id
    JOIN autores ON autores.id = libros_autores.autor_id
    LIMIT 5
''')
data =cursor.fetchall()

for a in data:
    print(a)